<a href="https://colab.research.google.com/github/AldiAlfatih/Program-Sistem-Cerdas-PTC/blob/main/PTC_Model_LSTM_dan_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn
!pip install tensorflow
!pip show tensorflow
!pip install --upgrade tensorflow
!pip install scikeras

Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tf_keras


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense
from sklearn.model_selection import RandomizedSearchCV
from scikeras.wrappers import KerasClassifier
from scipy.stats import randint
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 23.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.0
    Uninstalling scikit-learn-1.6.0:
      Successfully uninstalled scikit-learn-1.6.0


In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/Vinzzztty/playground-data-analyst/main/Machine%20Learning%20Terapan/2.%20Sentiment%20Analysis/dataset/dataset_selada.csv")

In [ ]:
df.head()

,Jam,temperature,humidity,light,pH,EC,TDS,WaterTemp,Label,Pattern,Kategori
0,15.15,25.5,67,17660,6.4,855,427,29.7,Selada,1,0
1,17.22,26.9,80,86930,6.7,929,464,27.0,Selada,1,0
2,7.33,25.7,77,23320,6.1,900,449,25.4,Selada,1,0
3,9.18,26.6,69,32280,6.7,887,443,26.1,Selada,1,0
4,10.39,25.8,65,41750,6.6,967,482,25.7,Selada,1,0


In [ ]:
# print(df['Jam'].dtype)
# # MEngubah tipe data Jam
# if df['Jam'].dtype != 'object':
#     df['Jam'] = df['Jam'].astype(str)
# # mengconvert 'JAM' menjadi 'menit'
# df['Jam'] = df['Jam'].str.split('.').apply(lambda x: int(x[0]) * 60 + int(float(x[1]) * 60 / 100))

In [ ]:
# Memilih fitur
fitur = ['temperature', 'humidity', 'TDS', 'WaterTemp']
X = df[fitur]

In [ ]:
X.head(10)

,temperature,humidity,TDS,WaterTemp
0,25.5,67,427,29.7
1,26.9,80,464,27.0
2,25.7,77,449,25.4
3,26.6,69,443,26.1
4,25.8,65,482,25.7
5,27.2,59,482,26.6
6,27.2,64,499,26.7
7,26.3,71,499,26.5
8,25.3,87,490,26.5
9,25.6,83,463,26.1


In [ ]:
# Menentukan jumlah cluster
n_clusters = 5

In [ ]:
# Melakukan clustering dengan K-Means
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df['Kategori'] = kmeans.fit_predict(X)
# kmeans = KMeans(n_clusters=n_clusters, random_state=42)
# df['Pattern'] = kmeans.fit_predict(X)

In [ ]:
# Membagi data
X = df[fitur]
y = df['Kategori']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# --- Model LSTM ---

# Penskalaan data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Transformasi data menjadi deret waktu
def create_dataset(dataset, labels, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :]
        X.append(a)
        Y.append(labels.iloc[i + look_back])
    X = np.array(X)
    Y = np.array(Y)
    X = np.pad(X, ((0, len(dataset) - len(X)), (0, 0), (0, 0)), 'edge')
    Y = np.pad(Y, (0, len(dataset) - len(Y)), 'edge')
    return X, Y

look_back = 10
X_train_lstm, y_train_lstm = create_dataset(X_train_scaled, y_train, look_back)
X_test_lstm, y_test_lstm = create_dataset(X_test_scaled, y_test, look_back)

# Reshape data untuk LSTM
X_train_lstm = np.reshape(X_train_lstm, (X_train_lstm.shape[0], X_train_lstm.shape[1], X_train_lstm.shape[2]))
X_test_lstm = np.reshape(X_test_lstm, (X_test_lstm.shape[0], X_test_lstm.shape[1], X_test_lstm.shape[2]))

# Fungsi untuk membuat layer LSTM dengan dropout
def create_lstm_layer(units, dropout_rate):
    return LSTM(units=units, return_sequences=False, dropout=dropout_rate)

# Definisikan model LSTM
def create_lstm_model(units=50, dropout_rate=0.2, optimizer='adam'):
    model = Sequential()
    model.add(Input(shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
    model.add(LSTM(units=units, return_sequences=True, dropout=dropout_rate))
    model.add(LSTM(units=units, return_sequences=False, dropout=dropout_rate))
    model.add(Dense(n_clusters, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Buat model LSTM dengan KerasClassifier
model_lstm = KerasClassifier(build_fn=create_lstm_model, verbose=0)

# Definisikan parameter distribution
param_dist_lstm = {
    'model__units': randint(30, 80),
    'batch_size': [16, 32, 64],
    'epochs': [10, 20, 30],
    'model__dropout_rate': [0.1, 0.2, 0.3],
    'model__optimizer': ['adam', 'rmsprop']
}

# Buat objek RandomizedSearchCV
random_search_lstm = RandomizedSearchCV(estimator=model_lstm, param_distributions=param_dist_lstm, n_iter=10, n_jobs=-1, cv=3)

# Latih model dengan RandomizedSearchCV
random_result_lstm = random_search_lstm.fit(X_train_lstm, y_train_lstm)

In [ ]:
#Menampilkan Hasil
print("Best LSTM Model: %f using %s" % (random_result_lstm.best_score_, random_result_lstm.best_params_))
best_lstm_model = random_result_lstm.best_estimator_.model_
best_lstm_model.save('best_model_lstm.h5')

#Menampilkan hasil prediksi pada data latih (LSTM)
y_pred_train_lstm = np.argmax(best_lstm_model.predict(X_train_lstm), axis=1)
print("Hasil prediksi pada data latih (LSTM):")
print(y_pred_train_lstm)

#Menampilkan hasil prediksi pada data uji (LSTM)
y_pred_lstm = np.argmax(best_lstm_model.predict(X_test_lstm), axis=1)
print("Hasil prediksi pada data uji (LSTM):")
print(y_pred_lstm)

#Evaluasi model LSTM pada data uji
print("Evaluasi Model LSTM:")
print(classification_report(y_test, y_pred_lstm))

Best LSTM Model: 0.420080 using {'batch_size': 32, 'epochs': 30, 'model__dropout_rate': 0.1, 'model__optimizer': 'rmsprop', 'model__units': 44}
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step
Hasil prediksi pada data latih (LSTM):
[2 2 2 ... 2 2 2]
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Hasil prediksi pada data uji (LSTM):
[2 2 2 ... 2 2 2]
Evaluasi Model LSTM:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       280
           1       0.00      0.00      0.00       165
           2       0.44      1.00      0.61       437
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00       118

    accuracy                           0.44      1001
   macro avg       0.09      0.20      0.12      1001
weighted avg       0.19      0.44      0.27      1001



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# --- Model ANN ---

def create_ann_model(neurons_1=128, neurons_2=64, activation='relu', optimizer='adam'):
    model = Sequential()
    model.add(Dense(neurons_1, activation=activation, input_dim=X_train_scaled.shape[1]))
    model.add(Dense(neurons_2, activation=activation))
    model.add(Dense(n_clusters, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Membangun model ANN dengan KerasClassifier
model_ann = KerasClassifier(build_fn=create_ann_model, verbose=0)
param_dist_ann = {
    'model__neurons_1': randint(64, 256),
    'model__neurons_2': randint(32, 128),
    'batch_size': [16, 32, 64],
    'epochs': [10, 20, 30],
    'model__activation': ['relu', 'sigmoid'],
    'model__optimizer': ['adam', 'rmsprop']
}

random_search_ann = RandomizedSearchCV(estimator=model_ann, param_distributions=param_dist_ann, n_iter=10, n_jobs=-1, cv=3)
random_result_ann = random_search_ann.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
#Menampilkan hasil
print("Best ANN Model: %f using %s" % (random_result_ann.best_score_, random_result_ann.best_params_))
best_ann_model = random_result_ann.best_estimator_.model_
best_ann_model.save('best_model_ann.h5')

#Menampilkan hasil prediksi pada data latih (ANN)
y_pred_train_ann = np.argmax(best_ann_model.predict(X_train_scaled), axis=1)
print("Hasil prediksi pada data latih (ANN):")
print(y_pred_train_ann)

#Menampilkan hasil prediksi pada data uji (ANN)
y_pred_ann = np.argmax(best_ann_model.predict(X_test_scaled), axis=1)
print("Hasil prediksi pada data uji (ANN):")
print(y_pred_ann)

#Evaluasi model ANN pada data uji
print("Evaluasi Model ANN:")
print(classification_report(y_test, y_pred_ann))

Best ANN Model: 0.993755 using {'batch_size': 32, 'epochs': 20, 'model__activation': 'relu', 'model__neurons_1': 130, 'model__neurons_2': 73, 'model__optimizer': 'adam'}
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Hasil prediksi pada data latih (ANN):
[4 4 0 ... 0 4 0]
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Hasil prediksi pada data uji (ANN):
[4 2 1 ... 0 1 0]
Evaluasi Model ANN:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       280
           1       1.00      1.00      1.00       165
           2       1.00      1.00      1.00       437
           3       1.00      1.00      1.00         1
           4       1.00      0.99      1.00       118

    accuracy                           1.00      1001
   macro avg       1.00      1.00      1.00      1001
weighted avg       1.00      1.00      1.00      1001



In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_ann)
print(cm)

[[280   0   0   0   0]
 [  0 165   0   0   0]
 [  2   0 435   0   0]
 [  0   0   0   1   0]
 [  0   0   1   0 117]]


In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('best_model_ann.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tfliteModel = converter.convert()

with open('best_model_ann.tflite', 'wb') as f:
    f.write(tfliteModel)

Saved artifact at '/tmp/tmpqcu0tng_'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 5), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  137442463362816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137442461647616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137442461655712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137442461660464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137442461657120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137442461893552: TensorSpec(shape=(), dtype=tf.resource, name=None)
